## Market Data:
    Preprocessing Development/Testing Scripts

There are thousands of files that all span different lengths of time and values. The ultimate goal is to know if the market is moving in a staistically significant positive or negative direction. In order to make that determination, each individual relavant entity must be known as moving in a significant direction. As a stock like Apple might move hundreds of dollars in a month and not be considered significant but a start up companyu might move the same amount and might be quite significant for it. To try and minimize the impact of individual stocks or companies having a significant price movement, only the ETF files will be processed as they group similar categories of companies together and reduce seen market volitility.

This file develops the import and processing of an ETF file. Attributes that need to be retained are the ETF name, the year, and if the price made a significant positive or negative movement. This process will be implemented into a loop in another script to process all the ETFs.

As this is a script to show the process the ETF data is going through, print statements are included however they are unneccessary during the actual processing.

In [409]:
import pandas as pd
import numpy as np
import os 

Obtain the file path and extract the ETF name

In [361]:
# assigns the relative file path. This will be an object instead of a named file path in the looped script
file = "Data/markets_test/aadr.us.txt"
#takes the string after the last '/'
etf_name = os.path.basename(os.path.normpath(file))
#removes the .txt of the file string
etf_name = etf_name.replace('.txt', '')
print(etf_name)

aadr.us


### Familiarize with data
  ###Not needed during looping script!!##

In [362]:
test_etf = pd.read_csv(file)

In [363]:
"Min", test_etf.min(), "Max", test_etf.max()

('Min',
 Date       2010-07-21
 Open           23.936
 High           23.946
 Low            23.867
 Close          23.946
 Volume              2
 OpenInt             0
 dtype: object,
 'Max',
 Date       2017-11-10
 Open            58.62
 High            58.72
 Low              57.7
 Close           58.43
 Volume         106139
 OpenInt             0
 dtype: object)

In [364]:
test_etf.Date.info()

<class 'pandas.core.series.Series'>
RangeIndex: 1565 entries, 0 to 1564
Series name: Date
Non-Null Count  Dtype 
--------------  ----- 
1565 non-null   object
dtypes: object(1)
memory usage: 12.4+ KB


In [365]:
test_etf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1565 entries, 0 to 1564
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Date     1565 non-null   object 
 1   Open     1565 non-null   float64
 2   High     1565 non-null   float64
 3   Low      1565 non-null   float64
 4   Close    1565 non-null   float64
 5   Volume   1565 non-null   int64  
 6   OpenInt  1565 non-null   int64  
dtypes: float64(4), int64(2), object(1)
memory usage: 85.7+ KB


#### Drop unneeded columns: OpenInt, Volume, Low, High

In [366]:
drop_columns = ["OpenInt", "Volume", "Low", "High"]
test_drop = test_etf.drop(drop_columns, axis = 1, inplace = True)

In [367]:
test_etf.head()

,Date,Open,Close
0,2010-07-21,24.333,23.946
1,2010-07-22,24.644,24.487
2,2010-07-23,24.759,24.507
3,2010-07-26,24.624,24.595
4,2010-07-27,24.477,24.517


In [368]:
# Add a column that shows sum of "Open" and "Close" for daily change

test_etf['change']= test_etf['Open'] - test_etf['Close']

In [369]:
test_etf.head()

,Date,Open,Close,change
0,2010-07-21,24.333,23.946,0.387
1,2010-07-22,24.644,24.487,0.157
2,2010-07-23,24.759,24.507,0.252
3,2010-07-26,24.624,24.595,0.029
4,2010-07-27,24.477,24.517,-0.040


#### Add the name of the ETF

In [370]:
test_etf['ETF_name'] = etf_name

#### Adjust the Date column for better processing

In [371]:
#convert sting to datetime format
test_etf['Date'] = pd.to_datetime(test_etf.Date) 

In [372]:
# seperate year, month and day
#test_etf['Year'], test_etf['Month'], test_etf['Day'] = test_etf.Date.dt.year, test_etf.Date.dt.month, test_etf.Date.dt.day
test_etf['Year'] = test_etf.Date.dt.year

In [373]:
#drop original date column
#test_etf.drop(columns = 'Date', axis =1, inplace = True)

In [374]:
test_etf.head()

,Date,Open,Close,change,ETF_name,Year
0,2010-07-21,24.333,23.946,0.387,aadr.us,2010
1,2010-07-22,24.644,24.487,0.157,aadr.us,2010
2,2010-07-23,24.759,24.507,0.252,aadr.us,2010
3,2010-07-26,24.624,24.595,0.029,aadr.us,2010
4,2010-07-27,24.477,24.517,-0.040,aadr.us,2010


#### Create Open and Close values for each Year

In [411]:
# create dataframe grouped by year
year_df = test_etf.groupby('Year')

In [412]:
# Find the first and last day of the year for each year
earliest_dates = year_df.Date.min()
last_dates = year_df.Date.max()

In [413]:
start_end_dates = test_etf.query("Date in @earliest_dates or Date in @last_dates")

In [438]:
start_end_dates

,Date,Open,Close,change,ETF_name,Year
0,2010-07-21,24.333,23.946,0.387,aadr.us,2010
112,2010-12-31,28.823,28.928,-0.105,aadr.us,2010
113,2011-01-03,29.065,29.141,-0.076,aadr.us,2011
309,2011-12-30,27.155,27.252,-0.097,aadr.us,2011
310,2012-01-03,28.087,27.766,0.321,aadr.us,2012
459,2012-12-31,30.107,29.853,0.254,aadr.us,2012
460,2013-01-02,30.741,30.593,0.148,aadr.us,2013
662,2013-12-31,37.059,36.532,0.527,aadr.us,2013
663,2014-01-02,36.921,36.356,0.565,aadr.us,2014
883,2014-12-31,36.687,36.687,0.000,aadr.us,2014


In [389]:
year = start_end_dates.Year.unique()

In [482]:
# create dataframe that to place needed values in
output_df = pd.DataFrame(columns = [ 'Year','ETF_name','Year_Open', 'Year_Close', 'Year_Change', 'Loss', 'Gain'])


#loop to generate new row in output database
for i in start_end_dates.Year.unique():

    #groups date data by year
    year_df = start_end_dates[start_end_dates.Year == i]

    #uses objects to reference needed values
    open = year_df.iloc[0].Open
    close = year_df.iloc[1].Close
    change = close - open
    
    # creates mapping array 
    new_row = {'Year':i, 'ETF_name':year_df.iloc[0].ETF_name, 'Year_Open':open, 'Year_Close':close, 'Year_Change':change}
    
    # inserts data into new row of the output dataframe
    output_df.loc[len(output_df)] = new_row

output_df['Loss'] = 'no'
output_df['Gain'] = 'no'

output_df

,Year,ETF_name,Year_Open,Year_Close,Year_Change,Loss,Gain
0,2010,aadr.us,24.333,28.928,4.595,no,no
1,2011,aadr.us,29.065,27.252,-1.813,no,no
2,2012,aadr.us,28.087,29.853,1.766,no,no
3,2013,aadr.us,30.741,36.532,5.791,no,no
4,2014,aadr.us,36.921,36.687,-0.234,no,no
5,2015,aadr.us,36.491,38.053,1.562,no,no
6,2016,aadr.us,39.941,39.916,-0.025,no,no
7,2017,aadr.us,40.711,56.400,15.689,no,no


#### Set upper and lower boundries for significant change
 This set any value over the 80 quantile or under the 20 quantile as notable

In [445]:
# sets the 80% and 20% boundries
etf_high_lim = output_df.Year_Change.quantile(.8)
etf_low_lim = output_df.Year_Change.quantile(.2)
etf_high_lim, etf_low_lim

(5.3126, -0.1504000000000033)

In [483]:
# compare the mean value of each year to the 20 and 80 limits previously set and 
# create a column documenting if it was outside the low/high lim

for i in range(len(output_df)):
    if output_df.Year_Change.iloc[i] < etf_low_lim:
        output_df.loc[i, 'Loss'] = 'yes'
        print('changed bad_year')
    elif output_df.Year_Change.iloc[i] > etf_high_lim:
        output_df.loc[i,'Gain'] = 'yes'
        print('changed good_year')
    else: 
        continue

changed bad_year
changed good_year
changed bad_year
changed good_year


In [484]:
output_df

,Year,ETF_name,Year_Open,Year_Close,Year_Change,Loss,Gain
0,2010,aadr.us,24.333,28.928,4.595,no,no
1,2011,aadr.us,29.065,27.252,-1.813,yes,no
2,2012,aadr.us,28.087,29.853,1.766,no,no
3,2013,aadr.us,30.741,36.532,5.791,no,yes
4,2014,aadr.us,36.921,36.687,-0.234,yes,no
5,2015,aadr.us,36.491,38.053,1.562,no,no
6,2016,aadr.us,39.941,39.916,-0.025,no,no
7,2017,aadr.us,40.711,56.400,15.689,no,yes
